## Big Data Analytics Project

### Group 7

##### Authors: Akshay Pradeep Bhala,  Juilee Prashant Salunkhe, Sai Praharsha Devalla, Yeswanth Reddy Velapalem

### Introduction:
Subject area is Social media Marketing and Content optimization.
<br>
About Data: <br>
Every online platform is striving to publish the articles on their site which have great value and bring most shares. In this project, we do the prediction of shares of an article based on the data produced by ‘Mashable’ where they collected data of around 39000 articles. Intend to help Mashable to decide which articles should they publish because they can actually predict which articles will be having the maximum number of shares. 
Project aims to do the following:
1. Descriptive analytics to further understand patterns, trends, and anomalies in data.
2. To understand and analysethe data and use neural-networks to classify the articles into different categories.
3. Binary classification as popular vs unpopular using a decision threshold of 1400 social interactions.the class value (shares) is continuously valued. We transformed the task into a binary task using a decision threshold of 1400.
4. Experiments with different models: Random Forest (best model),Adaboost, SVM, KNN and Naïve Bayes
5. Evaluate the performance of various models and select best two models for tuning process.

In [5]:
# Do not delete or change this cell

import os

# Define a function to determine if we are running on data bricks
# Return true if running in the data bricks environment, false otherwise
def is_databricks():
    # get the databricks runtime version
    db_env = os.getenv("DATABRICKS_RUNTIME_VERSION")
    
    # if running on data bricks
    if db_env != None:
        return True
    else:
        return False

# Define a function to read the data file.  The full path data file name is constructed
# by checking runtime environment variables to determine if the runtime environment is 
# databricks, or a student's personal computer.  The full path file name is then
# constructed based on the runtime env.
# 
# Params
#   data_file_name: The base name of the data file to load
# 
# Returns the full path file name based on the runtime env
#
def get_training_filename(data_file_name):    
    # if running on data bricks
    if is_databricks():
        # build the full path file name assuming data brick env
        full_path_name = "/FileStore/tables/%s" % data_file_name
    # else the data is assumed to be in the same dir as this notebook
    else:
        # Assume the student is running on their own computer and load the data
        # file from the same dir as this notebook
        full_path_name = data_file_name
    
    # return the full path file name to the caller
    return full_path_name

##### *Importing all the necessary libraries*

In [7]:
from pyspark.sql import functions as fn
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml import feature
from pyspark.ml import Pipeline
from pyspark.ml import regression
from pyspark.ml import classification
from math import exp,fabs
from pyspark.ml.feature import StandardScaler
import re 
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType,DoubleType
from pyspark.ml import feature, regression, evaluation, Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml import Pipeline
from sklearn.metrics import roc_curve
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator,RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.functions import when
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb
sb.set_palette('pastel')

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

##### *Loading the data*

In [9]:
pop_df = spark.read.csv(get_training_filename('OnlineNewsPopularity.csv'), header=True, inferSchema=True)
print(f"No of rows and columns in spam_df:({pop_df.count()},{len(pop_df.columns)})")
pop_df.toPandas().head(5)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593.0
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.148948,0.043137,0.015686,0.733333,0.266667,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711.0
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,918.0,918.0,918.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.323333,0.056872,0.009479,0.857143,0.142857,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500.0
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,4.404896,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028573,0.419300,0.494651,0.028905,0.028572,0.429850,0.100705,0.041431,0.020716,0.666667,0.333333,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200.0
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,4.682836,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,545.0,16000.0,3151.157895,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028633,0.028794,0.028575,0.028572,0.885427,0.513502,0.281003,0.074627,0.012127,0.860215,0.139785,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505.0


##### *Checking for the NaN and duplicate records*

In [11]:
#Checking for nan values
from pyspark.sql.functions import isnan, when, count, col
pop_df.select([count(when(isnan(c), c)).alias(c) for c in pop_df.columns]).toPandas().head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
#Checking duplicate values
pop_df.distinct().count()

Out[5]: 39644

Since all instances are distinct there are no duplicate values.

##### *Removing the spaces from the column names*

In [15]:
x = pop_df.columns 
for i in x:
    pop_df = pop_df.withColumnRenamed(i,i.replace(" ",""))

##### *We are removing the columns $url$ and $timedelta$ as the variance in data is 0*

In [17]:
colnames = ['timedelta','url']
pop_df = pop_df.drop(*colnames)
pop_df.toPandas().head()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,0.0,4.680365,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,496.0,496.0,496.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500331,0.378279,0.040005,0.041263,0.040123,0.521617,0.092562,0.045662,0.013699,0.769231,0.230769,0.378636,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593.0
1,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,0.0,4.913725,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.799756,0.050047,0.050096,0.050101,0.050001,0.341246,0.148948,0.043137,0.015686,0.733333,0.266667,0.286915,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711.0
2,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,0.0,4.393365,6.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,918.0,918.0,918.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.217792,0.033334,0.033351,0.033334,0.682188,0.702222,0.323333,0.056872,0.009479,0.857143,0.142857,0.495833,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500.0
3,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,0.0,4.404896,7.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028573,0.419300,0.494651,0.028905,0.028572,0.429850,0.100705,0.041431,0.020716,0.666667,0.333333,0.385965,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200.0
4,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,0.0,4.682836,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,545.0,16000.0,3151.157895,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028633,0.028794,0.028575,0.028572,0.885427,0.513502,0.281003,0.074627,0.012127,0.860215,0.139785,0.411127,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505.0


##### *Making a new column $target_shares$ to make the $shares$ into two classes. The two classes are seperated by a threshold value of 1400*

In [19]:
pop_df_final = pop_df.withColumn("target_shares", lit(0))
target_df = pop_df_final.withColumn("target_shares", \
              when(pop_df_final["shares"] < 1400,0).otherwise(1))

##### *Checking and removing outliers if any*

In [21]:
viz = target_df.toPandas()
til = viz.columns[:-2]

In [22]:
plt.figure(figsize=(15,50))
for i in range(len(til)):
    plt.subplot(10,6,i+1)
    plt.boxplot(viz[til[i]])
    plt.title(til[i])
display()

In [23]:
bounds = {
    c: dict(
        zip(["q1", "q3"], target_df.approxQuantile(c, [0.25, 0.75], 0))
    )
    for c in target_df.columns
}

In [24]:
for c in bounds:
    iqr = bounds[c]['q3'] - bounds[c]['q1']
    bounds[c]['lower'] = bounds[c]['q1'] - (iqr * 1.5)
    bounds[c]['upper'] = bounds[c]['q3'] + (iqr * 1.5)

In [25]:
bounds_df = pd.DataFrame(bounds)
bounds_df

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,kw_min_min,kw_max_min,kw_avg_min,kw_min_max,kw_max_max,kw_avg_max,kw_min_avg,kw_max_avg,kw_avg_avg,self_reference_min_shares,self_reference_max_shares,self_reference_avg_sharess,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend,LDA_00,LDA_01,LDA_02,LDA_03,LDA_04,global_subjectivity,global_sentiment_polarity,global_rate_positive_words,global_rate_negative_words,rate_positive_words,rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,target_shares
lower,4.5,-459.0,0.264107,1.0,0.432355,-11.0,-3.5,-3.5,-1.5,3.913742,1.5,0.0,0.0,0.0,0.0,0.0,0.0,-8.5,-387.5,-181.125,-11850.0,843300.0,-64376.25,-3085.070000,-124.671220,555.861416,-2302.5,-9250.0,-5347.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.298796,-0.163706,-0.429890,-0.492182,-0.528541,0.227910,-0.122360,-0.004458,-0.008570,0.3,-0.113960,0.148431,-0.025,-1.110223e-16,-0.540606,-1.3,-0.2375,-0.75,-0.225,-0.333333,-0.375,-1835.0,-1.5
q1,9.0,246.0,0.470860,1.0,0.625720,4.0,1.0,1.0,0.0,4.478400,6.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,445.0,141.750,0.0,843300.0,172837.50,0.000000,3562.101631,2382.432871,639.0,1100.0,981.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025050,0.025012,0.028571,0.028571,0.028574,0.396164,0.057754,0.028384,0.009615,0.6,0.185185,0.306229,0.050,6.000000e-01,-0.328385,-0.7,-0.1250,0.00,0.000,0.166667,0.000,946.0,0.0
q3,12.0,716.0,0.608696,1.0,0.754630,14.0,4.0,4.0,1.0,4.854839,9.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1000.0,357.000,7900.0,843300.0,330980.00,2056.713333,6019.950198,3600.147174,2600.0,8000.0,5200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.240948,0.150825,0.334212,0.375740,0.399984,0.508333,0.177829,0.050279,0.021739,0.8,0.384615,0.411428,0.100,1.000000e+00,-0.186905,-0.3,-0.0500,0.50,0.150,0.500000,0.250,2800.0,1.0
upper,16.5,1421.0,0.815449,1.0,0.947994,29.0,8.5,8.5,2.5,5.419497,13.5,0.0,0.0,0.0,0.0,0.0,0.0,11.5,1832.5,679.875,19750.0,843300.0,568193.75,5141.783333,9706.723049,5426.718630,5541.5,18350.0,11528.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.564795,0.339543,0.792674,0.896493,0.957099,0.676587,0.357943,0.083122,0.039925,1.1,0.683761,0.569226,0.175,1.600000e+00,0.025316,0.3,0.0625,1.25,0.375,1.000000,0.625,5581.0,2.5


##### *Winsorizing the outliers*

In [27]:
new_df=target_df
new_df=new_df.withColumn('n_tokens_title',when(new_df['n_tokens_title']>16.5,16.5).otherwise(new_df['n_tokens_title']))
new_df=new_df.withColumn('n_tokens_content',when(new_df['n_tokens_content']>1421,1421).otherwise(new_df['n_tokens_content']))
new_df=new_df.withColumn('n_unique_tokens',when(new_df['n_unique_tokens']>0.815449,0.815449).otherwise(new_df['n_unique_tokens']))
new_df=new_df.withColumn('n_non_stop_words',when(new_df['n_non_stop_words']>1.0,1.0).otherwise(new_df['n_non_stop_words']))
new_df=new_df.withColumn('n_non_stop_unique_tokens',when(new_df['n_non_stop_unique_tokens']>0.947994,0.947994).otherwise(new_df['n_non_stop_unique_tokens']))
new_df=new_df.withColumn('num_hrefs',when(new_df['num_hrefs']>29,29).otherwise(new_df['num_hrefs']))
new_df=new_df.withColumn('num_self_hrefs',when(new_df['num_self_hrefs']>8.5,8.5).otherwise(new_df['num_self_hrefs']))
new_df=new_df.withColumn('num_imgs',when(new_df['num_imgs']>8.5,8.5).otherwise(new_df['num_imgs']))
new_df=new_df.withColumn('num_videos',when(new_df['num_videos']>2.5,2.5).otherwise(new_df['num_videos']))
new_df=new_df.withColumn('average_token_length',when(new_df['average_token_length']>5.419497,5.419497).otherwise(new_df['average_token_length']))
new_df=new_df.withColumn('kw_min_min',when(new_df['kw_min_min']>11.5,11.5).otherwise(new_df['kw_min_min']))
new_df=new_df.withColumn('kw_max_min',when(new_df['kw_max_min']>1832.5,1832.5).otherwise(new_df['kw_max_min']))
new_df=new_df.withColumn('kw_avg_min',when(new_df['kw_avg_min']>679.875,679.875).otherwise(new_df['kw_avg_min']))
new_df=new_df.withColumn('kw_min_max',when(new_df['kw_min_max']>19750.0,19750.0).otherwise(new_df['kw_min_max']))
new_df=new_df.withColumn('kw_max_max',when(new_df['kw_max_max']>843300.0,843300.0).otherwise(new_df['kw_max_min']))
new_df=new_df.withColumn('kw_avg_max',when(new_df['kw_avg_max']>568193.75,568193.75).otherwise(new_df['kw_avg_max']))
new_df=new_df.withColumn('kw_min_avg',when(new_df['kw_min_avg']>5141.783333325,5141.783333325).otherwise(new_df['kw_min_avg']))
new_df=new_df.withColumn('kw_max_avg',when(new_df['kw_max_avg']>9706.723048789998,9706.723048789998).otherwise(new_df['kw_max_avg']))
new_df=new_df.withColumn('kw_avg_avg',when(new_df['kw_avg_avg']>5426.718629630001,5426.718629630001).otherwise(new_df['kw_avg_avg']))
new_df=new_df.withColumn('self_reference_min_shares',when(new_df['self_reference_min_shares']>5541.5,5541.5).otherwise(new_df['self_reference_min_shares']))
new_df=new_df.withColumn('self_reference_max_shares',when(new_df['self_reference_max_shares']>18350.0,18350.0).otherwise(new_df['self_reference_max_shares']))
new_df=new_df.withColumn('self_reference_avg_sharess',when(new_df['self_reference_avg_sharess']>11528.5,11528.5).otherwise(new_df['self_reference_avg_sharess']))
new_df=new_df.withColumn('LDA_00',when(new_df['LDA_00']>0.56479460105095,0.56479460105095).otherwise(new_df['LDA_00']))
new_df=new_df.withColumn('LDA_01',when(new_df['LDA_01']>0.3395427123779,0.3395427123779).otherwise(new_df['LDA_01']))
new_df=new_df.withColumn('LDA_02',when(new_df['LDA_02']>0.7926739292770502,0.7926739292770502).otherwise(new_df['LDA_02']))
new_df=new_df.withColumn('LDA_03',when(new_df['LDA_03']>0.89649332233195,0.89649332233195).otherwise(new_df['LDA_03']))
new_df=new_df.withColumn('LDA_04',when(new_df['LDA_04']>0.9570986642896501,0.9570986642896501).otherwise(new_df['LDA_04']))
new_df=new_df.withColumn('title_sentiment_polarity',when(new_df['title_sentiment_polarity']>0.375,0.375).otherwise(new_df['title_sentiment_polarity']))
new_df=new_df.withColumn('global_sentiment_polarity',when(new_df['global_sentiment_polarity']>0.3579429499083,0.3579429499083).otherwise(new_df['global_sentiment_polarity']))
new_df=new_df.withColumn('global_rate_positive_words',when(new_df['global_rate_positive_words']>0.08312190480824999,0.08312190480824999).otherwise(new_df['global_rate_positive_words']))
new_df=new_df.withColumn('global_rate_negative_words',when(new_df['global_rate_negative_words']>0.039924749163930004,0.039924749163930004).otherwise(new_df['global_rate_negative_words']))
new_df=new_df.withColumn('global_subjectivity',when(new_df['global_subjectivity']>0.6765873015865,0.6765873015865).otherwise(new_df['global_subjectivity']))
new_df=new_df.withColumn('rate_negative_words',when(new_df['rate_negative_words']>0.68376068376,0.68376068376).otherwise(new_df['rate_negative_words']))
new_df=new_df.withColumn('avg_negative_polarity',when(new_df['avg_negative_polarity']>0.02531622023800001,0.02531622023800001).otherwise(new_df['avg_negative_polarity']))
new_df=new_df.withColumn('min_negative_polarity',when(new_df['min_negative_polarity']>0.3,0.3).otherwise(new_df['min_negative_polarity']))
new_df=new_df.withColumn('abs_title_sentiment_polarity',when(new_df['abs_title_sentiment_polarity']>0.625,0.625).otherwise(new_df['abs_title_sentiment_polarity']))
new_df=new_df.withColumn('avg_positive_polarity',when(new_df['avg_positive_polarity']>0.5692255345665,0.5692255345665).otherwise(new_df['avg_positive_polarity']))
new_df=new_df.withColumn('min_positive_polarity',when(new_df['min_positive_polarity']>0.17500000000000002,0.17500000000000002).otherwise(new_df['min_positive_polarity']))
new_df=new_df.withColumn('n_tokens_title',when(new_df['n_tokens_title']<4.5,4.5).otherwise(new_df['n_tokens_title']))
new_df=new_df.withColumn('n_unique_tokens',when(new_df['n_unique_tokens']<0.2641073126304999,0.2641073126304999).otherwise(new_df['n_unique_tokens']))
new_df=new_df.withColumn('n_non_stop_unique_tokens',when(new_df['n_non_stop_unique_tokens']<0.43235498197850003,0.43235498197850003).otherwise(new_df['n_non_stop_unique_tokens']))
new_df=new_df.withColumn('average_token_length',when(new_df['average_token_length']<3.913741935479999,3.913741935479999).otherwise(new_df['average_token_length']))
new_df=new_df.withColumn('kw_avg_avg',when(new_df['kw_avg_avg']<555.86141587,555.86141587).otherwise(new_df['kw_avg_avg']))
new_df=new_df.withColumn('global_sentiment_polarity',when(new_df['global_sentiment_polarity']<-0.12236008039650,-0.12236008039650).otherwise(new_df['global_sentiment_polarity']))
new_df=new_df.withColumn('global_subjectivity',when(new_df['global_subjectivity']<0.2279100529105,0.2279100529105).otherwise(new_df['global_subjectivity']))
new_df=new_df.withColumn('rate_positive_words',when(new_df['rate_positive_words']<0.2999999999999999,0.2999999999999999).otherwise(new_df['rate_positive_words']))
new_df=new_df.withColumn('avg_positive_polarity',when(new_df['avg_positive_polarity']<0.148431009226499965,0.14843100922649996).otherwise(new_df['avg_positive_polarity']))
new_df=new_df.withColumn('avg_negative_polarity',when(new_df['avg_negative_polarity']<-0.54060639881,-0.54060639881).otherwise(new_df['avg_negative_polarity']))
new_df=new_df.withColumn('max_negative_polarity',when(new_df['max_negative_polarity']<-0.2375,-0.2375).otherwise(new_df['max_negative_polarity']))
new_df=new_df.withColumn('title_sentiment_polarity',when(new_df['title_sentiment_polarity']<-0.22499999999999998,-0.22499999999999998).otherwise(new_df['title_sentiment_polarity']))


### Exploratory Data Analysis and Visualization

##### Converting the weekday dummies into categorical variable

In [30]:
df = new_df.toPandas().copy()
day = df[['weekday_is_monday','weekday_is_tuesday','weekday_is_wednesday','weekday_is_thursday', 
          'weekday_is_friday','weekday_is_saturday' ,'weekday_is_sunday' ]]

In [31]:
temp_arr=[]
for r in list(range(day.shape[0])):
    for c in list(range(day.shape[1])):
        if ((c==0) and (day.iloc[r,c])==1):
            temp_arr.append('Monday')
        elif ((c==1) and (day.iloc[r,c])==1):
            temp_arr.append('Tuesday')
        elif ((c==2) and (day.iloc[r,c])==1):
            temp_arr.append('Wednesday')
        elif ((c==3) and (day.iloc[r,c])==1):
            temp_arr.append('Thursday')
        elif ((c==4) and (day.iloc[r,c])==1):
            temp_arr.append('Friday')
        elif ((c==5) and (day.iloc[r,c])==1):
            temp_arr.append('Saturday') 
        elif ((c==6) and (day.iloc[r,c])==1):
            temp_arr.append('Sunday')

##### Converting the data channel dummies into categorical variable

In [33]:
datachannel=df[['data_channel_is_lifestyle','data_channel_is_entertainment' ,'data_channel_is_bus','data_channel_is_socmed' ,
                'data_channel_is_tech', 'data_channel_is_world' ]]

In [34]:
datachannel_arr=[]
for r in list(range(datachannel.shape[0])):
    if (((datachannel.iloc[r,0])==0) and ((datachannel.iloc[r,1])==0) and ((datachannel.iloc[r,2])==0) and ((datachannel.iloc[r,3])==0) and ((datachannel.iloc[r,4])==0) and ((datachannel.iloc[r,5])==0)):
        datachannel_arr.append('Others')
    for c in list(range(datachannel.shape[1])):
        if ((c==0) and (datachannel.iloc[r,c])==1):
            datachannel_arr.append('Lifestyle')
        elif ((c==1) and (datachannel.iloc[r,c])==1):
            datachannel_arr.append('Entertainment')
        elif ((c==2) and (datachannel.iloc[r,c])==1):
            datachannel_arr.append('Business')
        elif ((c==3) and (datachannel.iloc[r,c])==1):
            datachannel_arr.append('Social Media')
        elif ((c==4) and (datachannel.iloc[r,c])==1):
            datachannel_arr.append('Tech')
        elif ((c==5) and (datachannel.iloc[r,c])==1):
            datachannel_arr.append('World')

In [35]:
df.insert(loc=11, column='weekdays', value=temp_arr)
df.insert(loc=12, column='data_channel', value=datachannel_arr)

# Now drop the old data
df.drop(labels=['data_channel_is_lifestyle','data_channel_is_entertainment' ,'data_channel_is_bus',
                        'data_channel_is_socmed' ,'data_channel_is_tech','data_channel_is_world', 
                 'weekday_is_monday','weekday_is_tuesday','weekday_is_wednesday', 
                      'weekday_is_thursday', 'weekday_is_friday','weekday_is_saturday' ,'weekday_is_sunday'], 
        axis = 1, inplace=True)

##### Target Variable count plot

In [37]:
plt.figure(figsize=(5,7))
sb.countplot(df['target_shares'])
plt.title('Distribution of target shares')
display()

##### Features

In [39]:
plt.figure(figsize=(30,15))

plt.subplot(1,2,1)
a=sb.countplot(df['weekdays'])#hue=df['target_shares'])
for p in a.patches:
        a.annotate(format(p.get_height()), (p.get_x() + p.get_width() / 2., 
        p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points', fontsize = 17)
plt.title('Weekdays',fontsize=15)
        
plt.subplot(1,2,2)
b=sb.countplot(df['data_channel'])#hue=df['target_shares'])
for p in b.patches:
        b.annotate(format(p.get_height()), (p.get_x() + p.get_width() / 2., 
        p.get_height()), ha = 'center', va = 'center', xytext = (0, 10), textcoords = 'offset points', fontsize = 17)
plt.title('Data Channel',fontsize=15)
display()

*As you can see, the weekdays have the highest number of shares. Wednesday, Tuesday, Thursday and Monday have the highest number of shares, with Friday having a significant drop.*

*We can also see which category or topic does the best. Here is a bar chart for the seven categories Tech, Entertainment, World, Business, Social Media, Lifestyle and Others.*

*The best performing category is World, followed by Tech, Entertainment, Business. The least popular categories are Social Media and Lifestyle.*

In [41]:
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
sb.boxplot(y=df['num_hrefs'],x=df['target_shares'])
plt.title('Num of links vs target shares')

plt.subplot(1,2,2)
sb.boxplot(y=df['num_self_hrefs'],x=df['target_shares'])
plt.title('Num of links to other Mashable articles vs target shares')
display()

In [42]:
plt.figure(figsize=(25,10))
plt.subplot(1,3,1)
sb.boxplot(y=df['self_reference_min_shares'],x=df['target_shares'])
plt.title('Minimum shares of Self referenced links')

plt.subplot(1,3,2)
sb.boxplot(y=df['self_reference_avg_sharess'],x=df['target_shares'])
plt.title('Average shares of Self referenced links')

plt.subplot(1,3,3)
sb.boxplot(y=df['self_reference_max_shares'],x=df['target_shares'])
plt.title('Maximum shares of Self referenced links')

display()

*When there are links embedded in the news article which point them to stories by other sources or referring to the news on their own website, seems to increase the popularity. If we look at both the references, the popularity increases when they refer the links from mashable compared to other online sources.*

In [44]:
plt.figure(figsize=(15,7))
plt.subplot(1,2,1)
sb.boxplot(y=df['global_sentiment_polarity'],x=df['target_shares'])
plt.title('global sentiment polarity vs target shares')

plt.subplot(1,2,2)
sb.boxplot(y=df['global_subjectivity'],x=df['target_shares'])
plt.title('global subjectivity vs target shares')

display()

**Polarity** *in sentiment analysis refers to identifying sentiment orientation (positive, neutral, and negative) in written or spoken language.*

**Subjectivity** *refers to the quality of being based on or influenced by personal feelings, tastes, or opinions.*


*The articles with more subjectivity and polarity tend to be more popular among the online news.*

In [46]:
plt.figure(figsize=(14,7))
sb.countplot(df['data_channel'],hue = df['target_shares'])
plt.title('Data Channel wrt target shares')
display()

**No. of words in title and content**

In [48]:
plt.figure(figsize=(30,10))
plt.subplot(1,3,1)
sb.countplot(df['n_tokens_title'],hue=df['target_shares'])
plt.title('Num of tokens in title wrt target shares')

plt.subplot(1,3,2)
sb.boxplot(y=df['n_tokens_content'],x=df['target_shares'])
plt.title('Num of tokens in content wrt target shares')

plt.subplot(1,3,3)
sb.boxplot(y=df['n_unique_tokens'],x=df['target_shares'])
plt.title('Num of unique tokens wrt target shares')
display()

**Num of keywords**

In [50]:
plt.figure(figsize=(14,7))
sb.countplot(df['num_keywords'],hue=df['target_shares'])
plt.title('Num of keywords wrt target shares')
display()

In [51]:
plt.figure(figsize=(20,20))
plt.subplot(3,1,1)
sb.boxplot(x=df['data_channel'],y=df['min_positive_polarity'])
plt.title("minimum positive polarity vs channel")

plt.subplot(3,1,2)
sb.boxplot(x=df['data_channel'],y=df['avg_positive_polarity'])
plt.title("average positive polarity vs channel")

plt.subplot(3,1,3)
sb.boxplot(x=df['data_channel'],y=df['max_positive_polarity'])
plt.title("maximum positive polarity vs channel")
display()

### Logistic Regression

##### Base Model

In [54]:
training_df, testing_df = new_df.randomSplit([0.7, 0.3],seed=42)
v = VectorAssembler().setInputCols(training_df.columns[:-2]).setOutputCol('cfeatures')
bc = BinaryClassificationEvaluator(labelCol='target_shares')
scale = feature.StandardScaler(withMean=True, withStd=False, inputCol='cfeatures', outputCol='sfeatures')
lr = LogisticRegression(featuresCol = 'sfeatures',labelCol = 'target_shares',regParam = 0.1)
lr_pipeline = Pipeline(stages=[v, scale, lr])
lr_fitted = lr_pipeline.fit(training_df)

In [55]:
print('The AUC Score: ',bc.evaluate(lr_fitted.transform(testing_df)))

The AUC Score: 0.7094810208352733

In [56]:
x = lr_fitted.transform(testing_df)

##### Metrics

In [58]:
evaluator = MulticlassClassificationEvaluator(labelCol='target_shares', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(x)
print("Accuracy = %g" % accuracy)
evaluatorf1 = MulticlassClassificationEvaluator(labelCol='target_shares', predictionCol='prediction', metricName='f1')
f1 = evaluatorf1.evaluate(x)
print("f1 = %g" % f1)
evaluatorwp = MulticlassClassificationEvaluator(labelCol='target_shares', predictionCol='prediction',
                                                metricName="weightedPrecision")
wp = evaluatorwp.evaluate(x)
print("Precision = %g" % wp)
evaluatorwr = MulticlassClassificationEvaluator(labelCol='target_shares', predictionCol='prediction', 
                                                metricName="weightedRecall")
wr = evaluatorwr.evaluate(x)
print("Recall = %g" % wr)

Accuracy = 0.655088
f1 = 0.654211
Precision = 0.654162
Recall = 0.655088

##### Grid Search

In [60]:
#lrparamGrid = ParamGridBuilder().addGrid(lr.regParam, [0.1,0.2]).addGrid(lr.elasticNetParam, [0.01,0.02]).build()
#lrcv = CrossValidator(estimator=lr_pipeline,
#                          estimatorParamMaps=lrparamGrid,
#                          evaluator=bc,
#                          numFolds=3)
#lrcvModel = lrcv.fit(training_df)

In [61]:
#lrpipe = lrcvModel.bestModel.stages[1]
#lrbest = lrpipe.extractParamMap()

#print(lrbest[lrpipe.getParam('regParam')])
#print(lrbest[lrpipe.getParam('elasticNetParam')])

##### Best Model

In [63]:
lr_best = LogisticRegression(featuresCol = 'sfeatures',labelCol = 'target_shares',regParam = 0.1,elasticNetParam = 0.01)
lr_pipeline_best = Pipeline(stages=[v,scale, lr_best])
lr_fitted_best = lr_pipeline_best.fit(training_df)

In [64]:
print('The AUC Score: ',bc.evaluate(lr_fitted_best.transform(testing_df)))

The AUC Score: 0.7094249018082257

In [65]:
x_best = lr_fitted_best.transform(testing_df)

##### Metrics

In [67]:
accuracy = evaluator.evaluate(x_best)
print("Accuracy = %g" % accuracy)
f1 = evaluatorf1.evaluate(x_best)
print("f1 = %g" % f1)
wp = evaluatorwp.evaluate(x_best)
print("Precision = %g" % wp)
wr = evaluatorwr.evaluate(x_best)
print("Recall = %g" % wr)

Accuracy = 0.654665
f1 = 0.653731
Precision = 0.65371
Recall = 0.654665

##### ROC Curve for Logistic Regression

In [69]:
fpr = lr_fitted_best.stages[2].summary.roc.select('FPR').collect()
tpr = lr_fitted_best.stages[2].summary.roc.select('TPR').collect()

plt.figure(figsize=(10, 6))
plt.scatter(fpr, tpr, linewidth=1, label='Best',color='green')
plt.plot([0, 1], [0, 1],linestyle='--', label='Random',color='blue')                                 
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16) 
plt.title('ROC for the Logistic Regression')
plt.legend()
plt.grid(True)  
display()

### Random Forest

##### Base Model

In [72]:
v = VectorAssembler().setInputCols(training_df.columns[:-2]).setOutputCol('cfeatures')
rf = RandomForestClassifier(featuresCol='cfeatures',labelCol='target_shares')
rf_pipe = Pipeline(stages=[v, rf])
rf_pipeline = rf_pipe.fit(training_df)

Out[40]: 0.7074002060456196

In [73]:
print('The AUC score: ',bc.evaluate(rf_pipeline.transform(testing_df)))

The AUC score: 0.707400206045619

In [74]:
lr_df = rf_pipeline.transform(testing_df)

##### Metrics

In [76]:
accuracy = evaluator.evaluate(lr_df)
print("Accuracy = %g" % accuracy)
f1 = evaluatorf1.evaluate(lr_df)
print("f1 = %g" % f1)
wp = evaluatorwp.evaluate(lr_df)
print("Precision = %g" % wp)
wr = evaluatorwr.evaluate(lr_df)
print("Recall = %g" % wr)

Accuracy = 0.650947
f1 = 0.647191
Precision = 0.650251
Recall = 0.650947

##### Grid Search

In [78]:
#rfparamGrid = (ParamGridBuilder().addGrid(rf.maxDepth, [3,6,9]).addGrid(rf.numTrees, [100,150]).addGrid(rf.featureSubsetStrategy,['log2','all','auto']).build())
#rfcv = CrossValidator(estimator = rf_pipe ,estimatorParamMaps = rfparamGrid, evaluator = bc, numFolds = 3)
#rfcvModel = rfcv.fit(training_df)

In [79]:
#bestPipeline = rfcvModel.bestModel.stages[1]
#bestParams = bestPipeline.extractParamMap()

#var_nam = ['maxDepth','numTrees','impurity','AUC Score']
#var_val = [bestParams[bestPipeline.getParam('maxDepth')],bestParams[bestPipeline.getParam('numTrees')],
#            bestParams[bestPipeline.getParam('featureSubsetStrategy')],0.8]
#var_val

##### Best Model

In [81]:
rf_best = RandomForestClassifier(featuresCol='cfeatures',labelCol='target_shares',numTrees=150,maxDepth=9,featureSubsetStrategy='auto')
rf_pipe_best = Pipeline(stages=[v, rf_best])
rf_pipeline_best = rf_pipe_best.fit(training_df)

In [82]:
print('The AUC Score: ',bc.evaluate(rf_pipeline_best.transform(testing_df)))

The AUC Score: 0.7255662339428963

In [83]:
t = rf_pipeline_best.transform(testing_df)

##### Metrics

In [85]:
evaluatorac = MulticlassClassificationEvaluator(labelCol='target_shares', predictionCol='prediction', metricName='accuracy')
accuracy = evaluatorac.evaluate(t)
print("Accuracy = %g" % accuracy)
evaluator_f = MulticlassClassificationEvaluator(labelCol='target_shares', predictionCol='prediction', metricName='f1')
f = evaluator_f.evaluate(t)
print("f1 = %g" % f)
evaluator_re = MulticlassClassificationEvaluator(labelCol='target_shares', predictionCol='prediction', metricName='weightedRecall')
re = evaluator_re.evaluate(t)
print("Recall = %g" % re)
evaluator_wp = MulticlassClassificationEvaluator(labelCol='target_shares', predictionCol='prediction', metricName='weightedPrecision')
pr = evaluator_wp.evaluate(t)
print("Precision = %g" % pr)

Accuracy = 0.665061
f1 = 0.663181
Recall = 0.665061
Precision = 0.664111

##### ROC Curve

In [87]:
preds = t.select('target_shares','probability').rdd.\
        map(lambda row: (float(row['probability'][1]), float(row['target_shares']))).collect()

y_score, y_true = zip(*preds)
fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label = 1)

plt.figure(figsize=(10, 6))
plt.scatter(fpr, tpr, linewidth=1, label='Best',color='green')
plt.plot([0, 1], [0, 1],linestyle='--', label='Random',color='blue')                                 
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16) 
plt.title('ROC for the Random Forest')
plt.legend()
plt.grid(True)  
display()

##### Feature Importance

In [89]:
bestModel = rf_pipeline_best.stages[1]
importances = bestModel.featureImportances

pd.DataFrame(list(zip(new_df.columns[:-2], bestModel.featureImportances.toArray())),
            columns = ['feature', 'importance']).sort_values('importance',ascending=False)

,feature,importance
25,kw_avg_avg,0.080578
26,self_reference_min_shares,0.066876
24,kw_max_avg,0.066696
36,is_weekend,0.044761
28,self_reference_avg_sharess,0.043160
12,data_channel_is_entertainment,0.039621
39,LDA_02,0.036206
23,kw_min_avg,0.032407
16,data_channel_is_world,0.028397
27,self_reference_max_shares,0.023281


In [90]:
x_values = list(range(len(importances)))
plt.figure(figsize=(10,5))
plt.bar(x_values, importances, orientation = 'vertical')
plt.xticks(x_values, training_df.columns[:-2], rotation=90)
plt.ylabel('Importance')
plt.xlabel('Feature')
plt.title('Feature Importance')
display()

## Gradient Boosting

##### Base Model

In [93]:
gbt = GBTClassifier(featuresCol='cfeatures',labelCol='target_shares')
gbt_pipe = Pipeline(stages=[v, gbt])
gbt_pipeline = gbt_pipe.fit(training_df)
bc = BinaryClassificationEvaluator(labelCol='target_shares')
prediction = gbt_pipeline.transform(testing_df)

In [94]:
print('The AUC Score: ',bc.evaluate(prediction))

The AUC Score: 0.7219886675196484

##### Metrics

In [96]:
evaluator = MulticlassClassificationEvaluator(labelCol='target_shares', metricName='accuracy')
accuracy = evaluator.evaluate(prediction)
print("Accuracy = %g" % accuracy)
evaluator_f = MulticlassClassificationEvaluator(labelCol='target_shares', metricName='f1')
f1 = evaluator_f.evaluate(prediction)
print("f1 = %g" % f1)
evaluator_re = MulticlassClassificationEvaluator(labelCol='target_shares', metricName='weightedRecall')
rec = evaluator_re.evaluate(prediction)
print("Recall = %g" % rec)
evaluator_wp = MulticlassClassificationEvaluator(labelCol='target_shares', metricName='weightedPrecision')
pre = evaluator_wp.evaluate(prediction)
print("Precision = %g" % pre)

Accuracy = 0.664554
f1 = 0.664036
Recall = 0.664554
Precision = 0.663888

##### Grid Search

In [98]:
#paramGrid = ParamGridBuilder()\
#            .addGrid(gbt.maxDepth, [2,5,9])\
#            .addGrid(gbt.featureSubsetStrategy, ['all'])\
#            .addGrid(gbt.minInstancesPerNode, [1,3])\
#            .addGrid(gbt.stepSize, [0.01,0.1,1]).build()
#crossval = CrossValidator(estimator=gbt_pipe, estimatorParamMaps=paramGrid, evaluator=bc, numFolds=3)

In [99]:
#model = crossval.fit(training_df)

In [100]:
#gb_bestpipe = model.bestModel.stages[1]
#gb_best = gb_bestpipe.extractParamMap()

#print(gb_best[gb_bestpipe.getParam('regParam')])
#print(gb_best[gb_bestpipe.getParam('elasticNetParam')])
#print("trained GBT classifier:%s" % model)
#prediction_gb = model.transform(testing_df)
#bc.evaluate(prediction_gb)

In [101]:
#var_val1 = [gb_best[gb_bestpipe.getParam('maxDepth')],gb_best[gb_bestpipe.getParam('minInstancesPerNode')],
#            gb_best[gb_bestpipe.getParam('featureSubsetStrategy')],gb_best[gb_bestpipe.getParam('stepSize')]]

#var_val1

##### Best Model

In [103]:
gbt_best = GBTClassifier(featuresCol='cfeatures',labelCol='target_shares',maxDepth = 5 ,minInstancesPerNode=1,featureSubsetStrategy='all',stepSize=0.1)
gbt_pipe_best  = Pipeline(stages=[v, gbt_best ])
gbt_pipeline_best  = gbt_pipe_best.fit(training_df)
bc = BinaryClassificationEvaluator(labelCol='target_shares')
prediction_best  = gbt_pipeline_best.transform(testing_df)

In [104]:
print('The AUC Score: ',bc.evaluate(prediction_best))

The AUC Score: 0.7219886675196485

##### Metrics

In [106]:
z = gbt_pipeline_best.transform(testing_df)
evaluatorac = MulticlassClassificationEvaluator(labelCol='target_shares', predictionCol='prediction', metricName='accuracy')
accuracy_gb = evaluatorac.evaluate(z)
print("Accuracy = %g" % accuracy_gb)
evaluator_f = MulticlassClassificationEvaluator(labelCol='target_shares', predictionCol='prediction', metricName='f1')
f1 = evaluator_f.evaluate(z)
print("f1 = %g" % f1)
evaluator_re = MulticlassClassificationEvaluator(labelCol='target_shares', predictionCol='prediction', metricName='weightedRecall')
rec = evaluator_re.evaluate(z)
print("Recall = %g" % rec)
evaluator_wp = MulticlassClassificationEvaluator(labelCol='target_shares', predictionCol='prediction', metricName='weightedPrecision')
pre = evaluator_wp.evaluate(z)
print("Precision = %g" % pre)

Accuracy = 0.664554
f1 = 0.664036
Recall = 0.664554
Precision = 0.663888

##### ROC Curve

In [108]:
preds = z.select('target_shares','probability').rdd.\
        map(lambda row: (float(row['probability'][1]), float(row['target_shares']))).collect()

y_score, y_true = zip(*preds)
fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label = 1)

plt.figure(figsize=(10, 6))
plt.scatter(fpr, tpr, linewidth=1, label='Best',color='green')
plt.plot([0, 1], [0, 1],linestyle='--', label='Random',color='blue')                                 
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16) 
plt.title('ROC for the Gradient Boosting Classifier')
plt.legend()
plt.grid(True)  
display()

### Random Forest Regression

##### *Base Model*

In [111]:
rf_rg = regression.RandomForestRegressor(featuresCol='cfeatures',labelCol='shares')
rf_rgpipe = Pipeline(stages=[v, rf_rg])
rf_rgpipe_fitted = rf_rgpipe.fit(training_df)
rf_df = rf_rgpipe_fitted.transform(testing_df)

##### *Finding the RMSE*

In [113]:
rf_df.select('cfeatures','shares','prediction')
evaluator = RegressionEvaluator(labelCol='shares',
                                predictionCol='prediction',
                                metricName='rmse')

rmse = evaluator.evaluate(rf_df)
print('Root Mean Squared Error (RMSE) on test data = %g' % rmse)

Root Mean Squared Error (RMSE) on test data = 13277.6

##### *Grid search for Random forest regressor*

In [115]:
#paramgrid_lin = ParamGridBuilder().addGrid(lin.numTrees, [100,200,300])\
#                .addGrid(lin.featureSubsetStrategy, ['auto','log2']).addGrid(lin.maxDepth, [1,3,5]).build()
#crossval_lin = CrossValidator(estimator=lin_pipeline,estimatorParamMaps=paramgrid_lin, 
#                              evaluator=RegressionEvaluator(labelCol='shares'), numFolds=3)
#cvmodel_lin = crossval_lin.fit(training_df)

In [116]:
#bestmodel_lin = cvmodel_lin.bestModel.stages[-1]
#bestParams_lin = bestmodel_lin.extractParamMap()

#var_nam_lin = ['numTrees','featureSubsetStrategy','maxDepth']
#var_val_lin = [bestParams_lin[bestmodel_lin.getParam('numTrees')],bestParams_lin[bestmodel_lin.getParam('featureSubsetStrategy')],
#               bestParams_lin[bestmodel_lin.getParam('maxDepth')]]

##### Model is built using the best parameter values from grid search

In [118]:
lin_t = regression.RandomForestRegressor(numTrees = 200, featureSubsetStrategy = 'log2', maxDepth = 5,
                                         featuresCol='cfeatures',labelCol='shares')
lin_t_pipeline = Pipeline(stages=[v, lin_t])
lin_t_fitted = lin_t_pipeline.fit(training_df)
lin_t_df = lin_t_fitted.transform(testing_df)

##### Finding RMSE for best model

In [120]:
evaluator = RegressionEvaluator(labelCol='shares',
                                predictionCol='prediction',
                                metricName='rmse')

rmse = evaluator.evaluate(lin_t_df)
print('Root Mean Squared Error (RMSE) on test data = %g' % rmse)

Root Mean Squared Error (RMSE) on test data = 12956